In [1]:
import datetime

# connect to the database mongodb
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

db = client['game_wiki']

In [2]:
# en general, lo que hay que havcer en cada funcion es una (o varias) querys por separado a la coleccion Hints, y luego juntarlas


# Es este caso, comprobamos que el usuario y la sala existan, y luego añadimos el comentario en la colección Hints, en vez de como se
# hacia antes, que se añadia tanto el usuario como la sala
def post_comment(db, user_email, room_id, text, category):
    # tenemos que verificar que el user_email exista
    user = db.users.find_one({"email": user_email})
    if not user:
        return {"status": "error", "message": "User does not exist"}
    
    # tennemos que verificar que la room_id exista
    room = db.rooms.find_one({"room_id": room_id})
    if not room:
        return {"status": "error", "message": "Room does not exist"}
    

    new_hint = {
        "Creation_date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "HintText": text,
        "Category": category,
        
        "References_room": {
            "IdR": room_id,
            "Name": room["room_name"],
            "IdD": room["dungeon_id"],
            "Dungeon": room["dungeon_name"]
        },

        "Publish_by": {
            "Email": user_email,
            "User_name": user["user_name"],
            "CreationDate": user["creation_date"],
            "Country": user["country"]
        }
    }
    
    db.Hints.insert_one(new_hint)

    return {"status": "success", "message": "Comment added"}








def get_dungeon_by_id(db, dungeon_id):
    # en este caso lo primero que hacer es buscar todas las hints que tengan como referencia el dungeon_id y que sean de tipo lore
    pipeline = [
        {"$match": {"References_room.IdD": dungeon_id, "Category": "lore"}},

        {"$group":{
            "_id": "$Category",
            "lore_hints": {"$push": {"lore_part": "$HintText"}}
        }}, 

        {"$project": {
            "_id": 0
        }}
    ]
    lore_part = list(db.Hints.aggregate(pipeline))


    # despues contamos cuantas hints de cada tipo hay en cada room de la dungeon
    pipeline = [
        {"$match": {"References_room.IdD": dungeon_id}},

        {"$group": {
            "_id": "$References_room.IdR",
            "bug": {"$sum": {"$cond": [{"$eq": ["$Category", "bug"]}, 1, 0]}},
            "hint": {"$sum": {"$cond": [{"$eq": ["$Category", "hint"]}, 1, 0]}},
            "lore": {"$sum": {"$cond": [{"$eq": ["$Category", "lore"]}, 1, 0]}},
            "suggestion": {"$sum": {"$cond": [{"$eq": ["$Category", "suggestion"]}, 1, 0]}},
        }}
    ]

    room_count_part = list(db.Hints.aggregate(pipeline))



    pipeline = [
        # Filtrar rooms por dungeon_id
        {"$match": {"dungeon_id": dungeon_id}},
        
        # Agrupar por dungeon_id para consolidar la información
        {"$group": {
            "_id": "$dungeon_id",
            "name": {"$first": "$dungeon_name"}, # la misma razón que en la función anterior
            "rooms": {"$push": {
                "room_id": "$room_id",
                "room_name": "$room_name",
                "connected_rooms": "$rooms_connected",
                "monsters": {
                    "$map": {  # Transforma cada monstruo en el array monsters para obtener solo id y nombre
                        "input": "$monsters",
                        "as": "monster",
                        "in": { # ponemos el doble $$ porque hacemos referencia a una variable de la variable
                            "id": "$$monster.id",
                            "name": "$$monster.name"
                        }
                    }
                },  # Tenemos que sacar aun el id y nombre de los monstruos
                "loots": {
                    "$map": {  # Transforma cada loot en el array loots para obtener solo id y nombre
                        "input": "$loot",
                        "as": "loot",
                        "in": {
                            "id": "$$loot.id",
                            "name": "$$loot.name"
                        }
                    }
                }  # Tenemos que sacar aun el id y nombre de los loots
                }
            }}
        },
        
        # Seleccionar los campos a mostrar
        {"$project": {
            "_id": 0,
            "id": "1",
            "name": 1,
            "rooms": 1
        }   
        }
    ]
    last_part = list(db.rooms.aggregate(pipeline))


    # añadir lore_part a last_part
    last_part[0]["lore_hints"] = lore_part[0]["lore_hints"]

    # añadir cada room_count_part a su respectiva room en last_part
    for room in last_part[0]["rooms"]:
        for roomc in room_count_part:
            if room["room_id"] == roomc["_id"]:
                room["bug"] = roomc["bug"]
                room["hint"] = roomc["hint"]
                room["lore"] = roomc["lore"]
                room["suggestion"] = roomc["suggestion"]
                break
            else:
                room["bug"] = 0
                room["hint"] = 0
                room["lore"] = 0
                room["suggestion"] = 0


    # vamos a hacer que loots sea un set para que no se repitan los loots
    for room in last_part[0]["rooms"]:
        if room["loots"]:
            room["loots"] = list({loot["id"]: loot for loot in room["loots"]}.values())

    return last_part







def get_room_by_id(bd, room_id):
    pipeline = [
        # Filtrar para obtener solo la habitación con el ID específico
        {"$match": {"room_id": room_id}},
        
        # Unir con la colección de monstruos si es necesario, asumiendo una colección separada
        {"$lookup": {
            "from": "monsters",
            "localField": "monsters",  # Asume un campo que contiene IDs de monstruos en la habitación
            "foreignField": "monster_id",
            "as": "monster_details"
        }},
        
        # Unir con la colección de tesoros si es necesario, asumiendo una colección separada
        {"$lookup": {
            "from": "loots",
            "localField": "loot",  # Asume un campo que contiene IDs de tesoros en la habitación
            "foreignField": "loot_id",
            "as": "loot_details"
        }},
        
        # Proyectar la información necesaria
        {"$project": {
            "_id": 0,
            "idR": "$room_id",
            "name": "$room_name",
            "inWP": "$in_waypoint",
            "outWP": "$out_waypoint",
            "monsters": {
                "$map": {
                    "input": "$monster_details",
                    "as": "monster",
                    "in": {
                        "idM": "$$monster.monster_id",
                        "name": "$$monster.name",
                        "type": "$$monster.type",
                        "level": "$$monster.level",
                        "place": "$$monster.place",
                        "exp": "$$monster.exp",
                        "manPage": "$$monster.manPage"
                    }
                }
            },
            "loots": {
                "$map": {
                    "input": "$loot_details",
                    "as": "loot",
                    "in": {
                        "idL": "$$loot.loot_id",
                        "name": "$$loot.name",
                        "type1": "$$loot.type1",
                        "type2": "$$loot.type2",
                        "weight": "$$loot.weight",
                        "gold": "$$loot.gold"
                    }
                }
            }
        }}
    ]

    room = list(db.rooms.aggregate(pipeline))

    # en este caso lo que hacemos es buscar todas las hints que tengan como referencia el room_id
    pipeline = [
        {"$match": {"References_room.IdR": room_id}},

        {"$project": {
            "_id": 0,
            "HintText": 1
        }}
        ]

    hints = list(db.Hints.aggregate(pipeline))

    room[0]["hints"] = hints


    return room





def user_by_email(db, email):
    # Coger un usuario por su email
    user = db.users.find_one({'email': email}, {'_id': 0})
    
    # buscar todas las hints que haya publicado el usuario
    pipeline = [
        {"$match": {"Publish_by.Email": {"$regex": email}}},

        {"$project": {"_id":0, "Publish_by":0}}
    ]

    hints = list(db.Hints.aggregate(pipeline))

    user["hints"] = hints

    return user